<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings

In [40]:
train_clean = pd.read_csv('train_clean.csv')
test_clean = pd.read_csv('test_clean.csv')

In [45]:
train_old = feather.read_dataframe('train_df')
test_old = feather.read_dataframe('test_df')

In [49]:
for df in [train_old, test_old]:
    for f in ['purchase_date_max','purchase_date_min','purchase_date_max_old',\
                     'purchase_date_min_old', 'first_active_month']:
        df[f] = df[f].astype(np.int64) * 1e-9

In [41]:
train_clean0 = train_clean.fillna(0)
train_clean0.to_feather('train_clean0')

In [4]:
def display_all(df):
    with pd.option_context("display.max_rows", 150, "display.max_columns", 100): 
        display(df)

In [9]:
display_all(train_clean.head().T)

,0,1,2,3,4
first_active_month,2017-06-01,2017-01-01,2016-08-01,2017-09-01,2017-11-01
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749
month,6,1,8,9,11
year,2017,2017,2016,2017,2017
elapsed_time,245,396,549,153,92
feature_1_1,0,0,0,0,1
feature_1_2,0,0,1,0,0
feature_1_3,0,0,0,0,0


In [10]:
display_all(test_clean.head().T)

,0,1,2,3,4
first_active_month,2017-04-01,2017-01-01,2017-08-01,2017-12-01,2015-12-01
card_id,C_ID_0ab67a22ab,C_ID_130fd0cbdd,C_ID_b709037bc5,C_ID_d27d835a9f,C_ID_2b5e3df5c2
feature_3,1,0,1,0,1
month,4,1,8,12,12
year,2017,2017,2017,2017,2015
elapsed_time,306,396,184,62,793
feature_1_1,0,0,0,0,0
feature_1_2,0,1,0,1,0
feature_1_3,1,0,0,0,0
feature_1_4,0,0,0,0,0


In [5]:
def print_score(m, x_trn, y_trn, x_val, y_val):
    res = [rmse(m.predict(x_trn), y_trn), rmse(m.predict(x_val), y_val),
                m.score(x_trn, y_trn), m.score(x_val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [6]:
def rmse(preds, target): 
    return np.sqrt(mean_squared_error(preds, target))

In [42]:
rf_args = dict({
    "n_estimators": 20, 
    "min_samples_leaf":3, 
    "max_features": 0.9, 
    "n_jobs":-1, 
    "oob_score":True
})

In [8]:
target = train_clean['target']

In [10]:
train_clean.size

24230040

In [13]:
train_clean.shape

(201917, 120)

In [14]:
train_df = train_clean.copy()

In [15]:
add_datepart(train_df, 'first_active_month')

In [16]:
display_all(train_df.dtypes)

card_id                                       object
feature_3                                      int64
month                                          int64
year                                           int64
elapsed_time                                   int64
feature_1_1                                    int64
feature_1_2                                    int64
feature_1_3                                    int64
feature_1_4                                    int64
feature_1_5                                    int64
feature_2_1                                    int64
feature_2_2                                    int64
feature_2_3                                    int64
authorized_transactions_count                  int64
authorized_category_1_sum                    float64
authorized_category_1_mean                   float64
authorized_category_3_A_mean                 float64
authorized_category_2_5.0_mean               float64
authorized_category_3_C_mean                 f

In [18]:
train_df.head()

,card_id,feature_3,month,year,elapsed_time,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,...,first_active_monthDay,first_active_monthDayofweek,first_active_monthDayofyear,first_active_monthIs_month_end,first_active_monthIs_month_start,first_active_monthIs_quarter_end,first_active_monthIs_quarter_start,first_active_monthIs_year_end,first_active_monthIs_year_start,first_active_monthElapsed
0,C_ID_92a2005557,1,6,2017,245,0,0,0,0,1,...,1,3,152,False,True,False,False,False,False,1496275200
1,C_ID_3d0044924f,0,1,2017,396,0,0,0,1,0,...,1,6,1,False,True,False,True,False,True,1483228800
2,C_ID_d639edf6cd,0,8,2016,549,0,1,0,0,0,...,1,0,214,False,True,False,False,False,False,1470009600
3,C_ID_186d6a6901,0,9,2017,153,0,0,0,1,0,...,1,4,244,False,True,False,False,False,False,1504224000
4,C_ID_cdbd2c0db2,0,11,2017,92,1,0,0,0,0,...,1,2,305,False,True,False,False,False,False,1509494400


In [19]:
train_df.drop('card_id', axis=1, inplace=True)

In [21]:
train_df2 = train_df.fillna(0)

In [43]:
train_df2.drop('outliers', axis=1, inplace=True)

In [22]:
train_cats(train_df2)

In [51]:
train_old.drop('card_id', axis=1, inplace=True)

In [53]:
df_hot_ext = feather.read_dataframe('df_hot_ext')
df_hot_test_ext = feather.read_dataframe('df_hot_test_ext')

In [55]:
for df in [df_hot_ext, df_hot_test_ext]:
    for f in ['purchase_date_amax','purchase_date_amin','purchase_date_amax_old',\
                     'purchase_date_amin_old']:
        df[f] = df[f].astype(np.int64) * 1e-9

In [62]:
df_hot_ext.drop('is_outlier', axis=1, inplace=True)

In [60]:
DataFrameSummary(df_hot_ext).summary().T

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
month_diff_mean_old,201917,11.7556,2.39709,10,10.5543,10.6744,11.6591,23,201917,32343,0,0%,numeric
category_1_sum_old,201917,6.37936,19.5725,0,0,0,5,1786,201917,357,0,0%,numeric
authorized_flag_mean_old,201917,0.893353,0.114446,0.0304878,0.852941,0.926829,0.972516,1,201917,11356,0,0%,numeric
purchase_amount_max,201917,-0.173108,1.45139,-0.746893,-0.634209,-0.51302,-0.251063,76.6829,201917,43853,0,0%,numeric
month_lag_mean_old,201917,-3.84099,2.12349,-12.4524,-5.52062,-3.57143,-2.03497,-0.0149254,201917,54905,0,0%,numeric
purchase_date_amax_old,201917,1.51585,0.00645884,1.48594,1.51475,1.5189,1.51967,1.51986,201917,186997,0,0%,numeric
category_1_mean_old,201917,0.10108,0.208051,0,0,0,0.0909091,1,201917,10370,0,0%,numeric
purchase_date_amin_old,201917,1.49475,0.0102942,1.48323,1.48434,1.49262,1.50419,1.51476,201917,193984,0,0%,numeric
purchase_amount_min_old,201917,-0.729993,0.0918995,-0.746908,-0.743902,-0.740296,-0.731881,28.5548,201917,7432,0,0%,numeric
hist_first_buy,201917,118.748,224.136,-331,10,22,123,1995,201917,1905,0,0%,numeric


In [63]:
m = RandomForestRegressor(**rf_args)
m.fit(df_hot_ext, target)
m.score(df_hot_ext, target)
print_score(m, df_hot_ext, target.values, df_hot_ext, target.values)

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:732: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


[2.303248285113751, 2.303248285113751, 0.6421925248673591, 0.6421925248673591, -0.023896062366220994]


- Base model - 0.8348232234871118, 0.8348232234871118, 0.9529936055779922, 0.9529936055779922, 0.8147359464393436
- Base model with outliers removed - 2.299946409218127, 2.299946409218127, 0.6432176762133195, 0.6432176762133195, -0.02364908316679548
- Base model with df_hot_ext - 2.1145840168101624, 2.1145840168101624, 0.6984094099753392, 0.6984094099753393, 0.1985261186716828
- Base model with df_hot_ext ouliers removed - 2.303248285113751, 2.303248285113751, 0.6421925248673591, 0.6421925248673591, -0.023896062366220994

In [34]:
display_all(train_clean.isna().sum().sort_values(ascending=False))

unauthorized_purchase_amount_std             51988
unauthorized_purchase_month_std              51988
unauthorized_installments_std                51988
new_installments_std                         48718
new_purchase_month_std                       48718
new_purchase_amount_std                      48718
unauthorized_month_lag_max                   30959
unauthorized_month_lag_min                   30959
unauthorized_category_2_3.0_mean             30959
unauthorized_installments_sum                30959
unauthorized_category_2_2.0_mean             30959
unauthorized_purchase_month_mean             30959
unauthorized_merchant_category_id_nunique    30959
unauthorized_purchase_date_ptp               30959
unauthorized_installments_min                30959
unauthorized_purchase_date_min               30959
unauthorized_installments_max                30959
unauthorized_installments_mean               30959
unauthorized_purchase_month_min              30959
unauthorized_category_2_4.0_mea

In [35]:
train_clean0 = train_clean.fillna(0)

In [37]:
train_clean0.isna().sum().sort_values(ascending=False)

outliers                                     0
new_city_id_nunique                          0
authorized_installments_mean                 0
authorized_installments_max                  0
authorized_installments_min                  0
authorized_installments_std                  0
authorized_merchant_category_id_nunique      0
authorized_purchase_month_mean               0
authorized_purchase_month_max                0
authorized_purchase_month_min                0
authorized_purchase_month_std                0
authorized_month_lag_min                     0
authorized_month_lag_max                     0
authorized_category_2_3.0_mean               0
authorized_category_2_2.0_mean               0
authorized_purchase_date_ptp                 0
authorized_purchase_date_min                 0
authorized_purchase_date_max                 0
authorized_category_3_B_mean                 0
authorized_city_id_nunique                   0
unauthorized_transactions_count              0
unauthorized_

In [38]:
train_clean0.to_feather('train_clean0')